# Unit 2 Assignment: Building a Mixture of Experts (MoE) Router

**Name:** Dhrushaj Achar
**SRN:** PES2UG23CS171
**Class:** 6C CSE

**Topic:** Advanced Architecture using Groq API
**Tools:** Python, Groq API, Dotenv

---

## Objective
Build a **Smart Customer Support Router** using a Mixture of Experts (MoE) architecture that routes user queries to specialized expert configurations (Technical, Billing, General) using LLM-based intent classification.

## 1. Install Required Libraries

In [5]:
!pip install groq python-dotenv -q


[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


## 2. Import Libraries and Initialize Groq Client

In [6]:
import os
import json
from groq import Groq
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize the Groq client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))
print("Groq client initialized successfully!")

Groq client initialized successfully!


## 3. Define Expert Configurations (`MODEL_CONFIG`)

We define different "experts" by using different **System Prompts** while using the same base model (`llama-3.3-70b-versatile`). Each expert specializes in a different domain:

- **Technical Expert** — Rigorous, code-focused, and precise
- **Billing Expert** — Empathetic, financial-focused, and policy-driven
- **General Expert** — Fallback for casual chat
- **Tool Expert** — Handles data lookup requests (Bonus)

In [ ]:
MODEL_CONFIG = {
    "technical": {
        "model": "llama-3.3-70b-versatile",
        "temperature": 0.7,
        "system_prompt": (
            "You are a Senior Technical Support Engineer. You are rigorous, code-focused, "
            "and precise. Your job is to diagnose bugs, provide exact code fixes, and explain "
            "technical concepts clearly. Always provide code snippets when relevant. "
            "Structure your responses with: 1) Problem Diagnosis, 2) Root Cause, 3) Solution/Fix. "
            "Use proper formatting for code blocks."
        )
    },
    "billing": {
        "model": "llama-3.3-70b-versatile",
        "temperature": 0.7,
        "system_prompt": (
            "You are a Billing & Accounts Support Specialist. You are empathetic, "
            "financial-focused, and policy-driven. You handle refund requests, subscription "
            "issues, payment disputes, and billing inquiries with care. Always acknowledge the "
            "customer's frustration, reference relevant policies, and provide clear next steps. "
            "Structure your responses with: 1) Acknowledgment, 2) Policy Reference, "
            "3) Resolution Steps."
        )
    },
    "general": {
        "model": "llama-3.3-70b-versatile",
        "temperature": 0.7,
        "system_prompt": (
            "You are a friendly and helpful General Support Assistant. You handle casual "
            "conversations, general inquiries, and anything that doesn't fall under technical "
            "or billing categories. Be warm, conversational, and helpful. If a query seems "
            "technical or billing-related, gently guide the user to clarify."
        )
    },
    "tool": {
        "model": "llama-3.3-70b-versatile",
        "temperature": 0.7,
        "system_prompt": (
            "You are a Data Lookup Assistant. You help users fetch real-time data such as "
            "cryptocurrency prices, stock prices, weather data, and other live information. "
            "When data is provided to you, present it clearly and concisely."
        )
    }
}

print("Expert configurations loaded:")
for expert, config in MODEL_CONFIG.items():
    print(f"  - {expert.capitalize()} Expert (model: {config['model']}, temp: {config['temperature']})")

Expert configurations loaded:
  - Technical Expert (model: mixtral-8x7b-32768, temp: 0.7)
  - Billing Expert (model: mixtral-8x7b-32768, temp: 0.7)
  - General Expert (model: mixtral-8x7b-32768, temp: 0.7)
  - Tool Expert (model: mixtral-8x7b-32768, temp: 0.7)


## 4. Build the Router Function (`route_prompt`)

The router is the **core** of the MoE architecture. It uses an LLM call with `temperature=0` (for deterministic, consistent classification) to classify user input into one of the expert categories.

The router returns **only** the category name — nothing else.

In [ ]:
def route_prompt(user_input):
    """
    Routes a user query to the appropriate expert category.
    
    Uses an LLM call with temperature=0 for deterministic classification.
    
    Args:
        user_input (str): The user's query string.
    
    Returns:
        str: The category name — one of 'technical', 'billing', 'tool', or 'general'.
    """
    
    VALID_CATEGORIES = ["technical", "billing", "tool", "general"]
    
    routing_prompt = (
        "You are a precise intent classifier. Classify the following user message "
        "into exactly ONE of these categories: [technical, billing, tool, general].\n\n"
        "Rules:\n"
        "- 'technical': Bug reports, code errors, programming questions, software issues.\n"
        "- 'billing': Refund requests, payment issues, subscription problems, charges.\n"
        "- 'tool': Requests for live/real-time data like prices (crypto, stocks), weather, etc.\n"
        "- 'general': Casual chat, greetings, or anything that doesn't fit above.\n\n"
        "Return ONLY the single category word. No explanation, no punctuation, no extra text."
    )
    
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": routing_prompt},
            {"role": "user", "content": user_input}
        ],
        temperature=0,
        max_tokens=10
    )
    
    category = response.choices[0].message.content.strip().lower()
    
    # Validate the category; default to 'general' if unrecognized
    if category not in VALID_CATEGORIES:
        print(f"  [Router Warning] Unrecognized category '{category}', defaulting to 'general'.")
        category = "general"
    
    return category

print("Router function defined successfully!")

Router function defined successfully!


## 5. Bonus: Mock Tool Function for Data Fetching

Before building the orchestrator, we define the mock tool function that simulates fetching live data (e.g., cryptocurrency prices). This is used when the router classifies a query as `tool`.

In [9]:
def mock_fetch_price(asset_name):
    """
    Simulates fetching real-time price data for an asset.
    
    Args:
        asset_name (str): The name of the asset (e.g., 'bitcoin', 'ethereum').
    
    Returns:
        str: A formatted string with the mock price data.
    """
    
    # Mock database of asset prices
    mock_prices = {
        "bitcoin": {"price": "$67,432.15", "change_24h": "+2.3%", "market_cap": "$1.32T"},
        "ethereum": {"price": "$3,521.80", "change_24h": "+1.8%", "market_cap": "$423B"},
        "solana": {"price": "$142.67", "change_24h": "-0.5%", "market_cap": "$63B"},
        "dogecoin": {"price": "$0.1234", "change_24h": "+5.1%", "market_cap": "$17.6B"},
    }
    
    asset_key = asset_name.lower().strip()
    
    if asset_key in mock_prices:
        data = mock_prices[asset_key]
        return (
            f"📊 {asset_name.capitalize()} Price Data (Mock):\n"
            f"  Price: {data['price']}\n"
            f"  24h Change: {data['change_24h']}\n"
            f"  Market Cap: {data['market_cap']}"
        )
    else:
        return f"⚠️ Sorry, price data for '{asset_name}' is not available in our mock database."


def extract_asset_from_query(user_input):
    """
    Extracts the asset name from a user's price query using simple keyword matching.
    """
    known_assets = ["bitcoin", "ethereum", "solana", "dogecoin"]
    user_lower = user_input.lower()
    
    for asset in known_assets:
        if asset in user_lower:
            return asset
    
    # If no known asset found, try to extract from common patterns
    words = user_lower.split()
    for i, word in enumerate(words):
        if word in ["of", "for"] and i + 1 < len(words):
            candidate = words[i + 1].strip("?.,!")
            return candidate
    
    return "unknown"

print("Mock tool functions defined successfully!")

Mock tool functions defined successfully!


## 6. Build the Orchestrator Function (`process_request`)

The orchestrator ties everything together:
1. Calls `route_prompt()` to classify the user's intent
2. If the category is `tool`, calls the mock data-fetching function
3. Otherwise, selects the correct expert's System Prompt and calls the LLM
4. Returns the expert's response

In [10]:
def process_request(user_input):
    """
    Main orchestrator function that processes a user request through the MoE pipeline.
    
    1. Routes the query to the appropriate expert category.
    2. If 'tool', calls the mock data-fetching function.
    3. Otherwise, calls the LLM with the expert's system prompt.
    4. Returns the final response.
    
    Args:
        user_input (str): The user's query string.
    
    Returns:
        str: The expert's response.
    """
    
    print(f"\n{'='*60}")
    print(f"📨 User Query: {user_input}")
    print(f"{'='*60}")
    
    # Step 1: Route the query
    category = route_prompt(user_input)
    print(f"🔀 Router Decision: '{category}' expert selected")
    print(f"{'-'*60}")
    
    # Step 2: Handle tool requests (Bonus)
    if category == "tool":
        asset_name = extract_asset_from_query(user_input)
        print(f"🔧 Tool Use: Fetching data for '{asset_name}'...")
        result = mock_fetch_price(asset_name)
        print(f"\n{result}")
        return result
    
    # Step 3: Get the expert configuration
    expert_config = MODEL_CONFIG.get(category, MODEL_CONFIG["general"])
    
    # Step 4: Call the LLM with the expert's system prompt
    response = client.chat.completions.create(
        model=expert_config["model"],
        messages=[
            {"role": "system", "content": expert_config["system_prompt"]},
            {"role": "user", "content": user_input}
        ],
        temperature=expert_config["temperature"],
        max_tokens=1024
    )
    
    expert_response = response.choices[0].message.content
    
    print(f"🤖 {category.capitalize()} Expert Response:\n")
    print(expert_response)
    
    return expert_response

print("Orchestrator function defined successfully!")

Orchestrator function defined successfully!


---

## 7. Testing the MoE Router

### Test 1: Technical Query
The router should classify this as **"technical"** and respond with code-focused debugging advice.

In [11]:
# Test 1: Technical Query
response = process_request("My python script is throwing an IndexError on line 5.")


📨 User Query: My python script is throwing an IndexError on line 5.


BadRequestError: Error code: 400 - {'error': {'message': 'The model `mixtral-8x7b-32768` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}

### Test 2: Billing Query
The router should classify this as **"billing"** and respond with empathetic, policy-driven support.

In [ ]:
# Test 2: Billing Query
response = process_request("I was charged twice for my subscription this month.")

### Test 3: General Query
The router should classify this as **"general"** and respond with friendly casual conversation.

In [ ]:
# Test 3: General Query
response = process_request("Hey, how are you doing today?")

### Test 4: Bonus — Tool Use Expert (Bitcoin Price Query)
The router should classify this as **"tool"** and return mock-fetched Bitcoin price data instead of an LLM-generated response.

In [ ]:
# Test 4: Bonus - Tool Use Expert (Bitcoin Price)
response = process_request("What is the current price of Bitcoin?")